In [9]:
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")


from chart_studio.plotly import plotly as py
import plotly.graph_objs as go

#import Keras
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

#read the data in csv
df_sales = pd.read_csv('transactions_22104.csv')

#convert date field from string to datetime
df_sales['date'] = pd.to_datetime(df_sales['date'])

#represent month in date field as its first day
df_sales['date'] = df_sales['date'].dt.year.astype('str') + '-' + df_sales['date'].dt.month.astype('str') + '-01'
df_sales['date'] = pd.to_datetime(df_sales['date'])

#groupby date and sum the sales
df_sales = df_sales.groupby('date').sales.sum().reset_index()

#create a new dataframe to model the difference ( Using Seasonal difference of 12 months)
df_diff = df_sales.copy()

#add previous sales to the next row
df_diff['prev_sales'] = df_diff['sales'].shift(12)

#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['sales'] - df_diff['prev_sales'])

#create dataframe for transformation from time series to supervised
df_supervised = df_diff.drop(['prev_sales'],axis=1)

#adding lags
for inc in range(1,13):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)

# #drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)

#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['sales','date'],axis=1)

#split train and test set
data_set_length = len(df_model)
test_set_length = int (.25 * len(df_model))
train_set, test_set = df_model[0:-test_set_length].values, df_model[-test_set_length:].values

#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)

# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)

# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [10]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, nb_epoch=150, batch_size=1, verbose=1, shuffle=False)

Epoch 1/150
14/14 [==============================] - 0s 22ms/step - loss: 0.4719
Epoch 2/150
14/14 [==============================] - 0s 2ms/step - loss: 0.4733
Epoch 3/150
14/14 [==============================] - 0s 2ms/step - loss: 0.4349
Epoch 4/150
14/14 [==============================] - 0s 2ms/step - loss: 0.4019
Epoch 5/150
14/14 [==============================] - 0s 2ms/step - loss: 0.3727
Epoch 6/150
14/14 [==============================] - 0s 2ms/step - loss: 0.3469
Epoch 7/150
14/14 [==============================] - 0s 2ms/step - loss: 0.3239
Epoch 8/150
14/14 [==============================] - 0s 2ms/step - loss: 0.3034
Epoch 9/150
14/14 [==============================] - 0s 2ms/step - loss: 0.2850
Epoch 10/150
14/14 [==============================] - 0s 3ms/step - loss: 0.2685
Epoch 11/150
14/14 [==============================] - 0s 2ms/step - loss: 0.2535
Epoch 12/150
14/14 [==============================] - 0s 2ms/step - loss: 0.2398
Epoch 13/150
14/14 [================

14/14 [==============================] - 0s 2ms/step - loss: 0.0132
Epoch 103/150
14/14 [==============================] - 0s 2ms/step - loss: 0.0130
Epoch 104/150
14/14 [==============================] - 0s 2ms/step - loss: 0.0127
Epoch 105/150
14/14 [==============================] - 0s 2ms/step - loss: 0.0124
Epoch 106/150
14/14 [==============================] - 0s 2ms/step - loss: 0.0121
Epoch 107/150
14/14 [==============================] - 0s 1ms/step - loss: 0.0119
Epoch 108/150
14/14 [==============================] - 0s 2ms/step - loss: 0.0116
Epoch 109/150
14/14 [==============================] - 0s 2ms/step - loss: 0.0114
Epoch 110/150
14/14 [==============================] - 0s 2ms/step - loss: 0.0112
Epoch 111/150
14/14 [==============================] - 0s 2ms/step - loss: 0.0109
Epoch 112/150
14/14 [==============================] - 0s 2ms/step - loss: 0.0107
Epoch 113/150
14/14 [==============================] - 0s 2ms/step - loss: 0.0105
Epoch 114/150
14/14 [=========

In [29]:
y_pred = model.predict(X_test,batch_size=1)
X_test

array([[[-0.98877859, -0.82046708, -1.22978837, -1.84542378,
         -1.29154423, -1.21156952, -1.06453653, -1.09136231,
         -1.29866772, -1.06624616, -0.71909377,  0.2926832 ]],

       [[-0.9773101 , -0.9904872 , -1.02962816, -2.17234643,
         -2.26020723, -1.53395469, -1.28293304, -1.15884695,
         -1.40367351, -1.37480668, -1.06624616, -0.3405696 ]],

       [[ 0.10575375, -0.98076494, -1.2191826 , -1.88757612,
         -2.63478608, -2.60508747, -1.61572102, -1.38722006,
         -1.48123589, -1.48329059, -1.37480668, -0.61128313]],

       [[-0.24442322, -0.06261187, -1.20834331, -2.15725748,
         -2.30850421, -3.01929106, -2.7214174 , -1.73521017,
         -1.74371284, -1.56342207, -1.48329059, -0.85190226]]])

In [25]:

#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])

#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df_sales[-test_set_length:].date)
actual_sales = list(df_sales[-test_set_length- 12:].sales)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    
    result_dict['pred_value'] = int(pred_test_set_inverted[index][1] + actual_sales[index])
    result_dict['date'] = sales_dates[index]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)


(1, 1, 1)

In [13]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df_sales,df_result,on='date',how='left')
#plot actual and predicted
plot_data = [
    go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['sales'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['pred_value'],
        name='predicted'
    )   
]
plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)

fig.show()